In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date

## Reading in the data

In [2]:
#read in the data
mid = pd.read_csv("MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
mid.drop([0,1], inplace=True)
print(mid.describe())
#bring the time data into a new df
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
print(time.describe())
#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    masses = [2,4,5.5,12,15,16,18,28,32,40,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp-timestamp[2]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time)

#time["Elapsed time, mass 40"].describe()

       mass 2.0_value  mass 4.0_value  mass 5.5_value  mass 12.0_value  \
count    1.697000e+03    1.697000e+03    1.697000e+03     7.600000e+02   
mean     2.037199e-10    4.601856e-13    5.874125e-14     7.766868e-14   
std      5.271121e-10    2.251720e-13    6.557085e-14     2.285582e-13   
min      2.820090e-11    0.000000e+00    1.800000e-15     0.000000e+00   
25%      4.140030e-11    2.060000e-13    2.040000e-14     6.675000e-15   
50%      5.151870e-11    5.804000e-13    4.270000e-14     1.470000e-14   
75%      2.524410e-10    6.435000e-13    6.420000e-14     3.325000e-14   
max      5.392830e-09    7.842000e-13    5.939000e-13     1.490600e-12   

       mass 15.0_value  mass 16.0_value  mass 18.0_value  mass 28.0_value  \
count     5.000000e+01     7.000000e+00     5.000000e+00     5.000000e+00   
mean      4.224200e-13     6.674286e-14     3.599200e-13     2.245600e-13   
std       2.209199e-13     7.664065e-14     5.783759e-14     6.517793e-14   
min       1.920000e-14   

,mass 2.0_time,mass 4.0_time,mass 5.5_time,mass 12.0_time,mass 15.0_time,mass 16.0_time,mass 18.0_time,mass 28.0_time,mass 32.0_time,mass 40.0_time,...,"Elapsed time, mass 4","Elapsed time, mass 5.5","Elapsed time, mass 12","Elapsed time, mass 15","Elapsed time, mass 16","Elapsed time, mass 18","Elapsed time, mass 28","Elapsed time, mass 32","Elapsed time, mass 40","Elapsed time, mass 44"
2,2023/01/13 14:48:15.864,2023/01/13 14:48:19.115,2023/01/13 14:48:19.365,2023/01/13 14:48:19.615,2023/01/13 14:48:19.866,2023/01/13 14:48:20.116,2023/01/13 14:48:20.366,2023/01/13 14:48:20.616,2023/01/13 14:48:20.866,2023/01/13 14:48:21.116,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,NaN
3,2023/01/13 14:48:22.260,2023/01/13 14:48:25.511,2023/01/13 14:48:25.761,2023/01/13 14:48:26.011,2023/01/13 14:48:26.261,2023/01/13 14:48:26.511,2023/01/13 14:48:26.761,2023/01/13 14:48:27.012,2023/01/13 14:48:27.262,2023/01/13 14:48:27.512,...,6.396,6.396,6.396,6.395,6.395,6.395,6.396,6.396,6.396,NaN
4,2023/01/13 14:48:28.655,2023/01/13 14:48:31.906,2023/01/13 14:48:32.156,2023/01/13 14:48:32.406,2023/01/13 14:48:32.656,2023/01/13 14:48:32.906,2023/01/13 14:48:33.156,2023/01/13 14:48:33.407,2023/01/13 14:48:33.657,2023/01/13 14:48:33.907,...,12.791,12.791,12.791,12.790,12.790,12.790,12.791,12.791,12.791,NaN
5,2023/01/13 14:48:35.050,2023/01/13 14:48:38.301,2023/01/13 14:48:38.551,2023/01/13 14:48:38.801,2023/01/13 14:48:39.051,2023/01/13 14:48:39.301,2023/01/13 14:48:39.551,2023/01/13 14:48:39.802,2023/01/13 14:48:40.052,2023/01/13 14:48:40.302,...,19.186,19.186,19.186,19.185,19.185,19.185,19.186,19.186,19.186,NaN
6,2023/01/13 14:48:41.445,2023/01/13 14:48:44.696,2023/01/13 14:48:44.946,2023/01/13 14:48:45.196,2023/01/13 14:48:45.446,2023/01/13 14:48:45.696,2023/01/13 14:48:45.946,2023/01/13 14:48:46.197,2023/01/13 14:48:46.447,NaN,...,25.581,25.581,25.581,25.580,25.580,25.580,25.581,25.581,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1694,2023/01/13 17:48:39.246,2023/01/13 17:48:39.496,2023/01/13 17:48:40.747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10820.381,10821.382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1695,2023/01/13 17:48:45.641,2023/01/13 17:48:45.891,2023/01/13 17:48:47.142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10826.776,10827.777,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1696,2023/01/13 17:48:52.036,2023/01/13 17:48:52.286,2023/01/13 17:48:53.537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10833.171,10834.172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1697,2023/01/13 17:48:58.431,2023/01/13 17:48:58.681,2023/01/13 17:48:59.932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10839.566,10840.567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#reading the Emission current, pressures
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()
#adding an elapsed time column
timestamp=[]
for i in vaclog["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
vaclog.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
1,NaN,13/01/2023 09:37:08,8.0,0.006098,0.0,0.0,0.000002,4.539000e-07,0.22708,0.0,-25.589,195.397,NaN,NaN
2,NaN,13/01/2023 09:37:15,15.0,0.006244,0.0,0.0,0.000002,4.544000e-07,0.22708,0.0,-25.589,195.285,NaN,NaN
3,NaN,13/01/2023 09:37:22,22.0,0.005826,0.0,0.0,0.000002,4.539000e-07,0.22708,0.0,-25.589,195.397,NaN,NaN
4,NaN,13/01/2023 09:37:29,29.0,0.005948,0.0,0.0,0.000002,4.544000e-07,0.22709,0.0,-25.926,195.397,NaN,NaN
5,NaN,13/01/2023 09:37:36,36.0,0.006393,0.0,0.0,0.000002,4.544000e-07,0.22708,0.0,-25.814,195.285,NaN,NaN


In [4]:
#Reading in the HiVolta measurement file 
hv = pd.read_csv("hv log", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["em"]=[element * 1e-6 for element in hv["IMon2"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
#hv["em"]=abs(hv["em"])
#hv["hv_grid"]=abs(hv["hv_grid"])
hv["datetime"] = pd.to_datetime(hv['Date'] + ' ' + hv['Time'],format="%d/%m/%Y %H:%M:%S.%f")
hv['datetime'] = hv['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')
#hv.insert(0,"datetime",hv["Date"]+" "+hv["Time"])

#adding an elapsed time column
timestamp=[]
for i in hv["Time"]:
    timestamp.append(i)
    
#making the column into a datetime object
timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)


hv



,Date,Time,Elapsed time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,VMon7,...,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,Comment,hv_grid,em,datetime
0,13/01/2023,14:50:55.409,0.000,0.02,0.06,0.04,0.08,0.00,0.00,0.0,...,0.0087,0.0080,0.0029,0.0090,-0.0216,-0.0125,NaN,-9.000000e-10,8.100000e-09,2023-01-13 14:50:55.409000
1,13/01/2023,14:50:56.594,1.185,0.02,0.06,0.04,0.08,0.00,0.00,0.0,...,0.0087,0.0080,0.0026,0.0090,-0.0216,-0.0125,NaN,-9.000000e-10,8.100000e-09,2023-01-13 14:50:56.594000
2,13/01/2023,14:50:57.767,2.358,0.02,0.06,0.04,0.08,0.00,0.00,0.0,...,0.0087,0.0080,0.0023,0.0090,-0.0216,-0.0125,NaN,-9.000000e-10,8.100000e-09,2023-01-13 14:50:57.767000
3,13/01/2023,14:50:58.953,3.544,0.02,0.06,0.04,0.08,0.00,0.00,0.0,...,0.0087,0.0080,0.0021,0.0090,-0.0216,-0.0125,NaN,-9.000000e-10,8.100000e-09,2023-01-13 14:50:58.953000
4,13/01/2023,14:51:00.135,4.726,0.02,0.06,0.04,0.08,0.00,0.00,0.0,...,0.0087,0.0080,0.0020,0.0090,-0.0216,-0.0125,NaN,-9.000000e-10,8.100000e-09,2023-01-13 14:51:00.135000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8929,13/01/2023,17:49:34.461,10719.052,499.96,100.08,1000.02,1000.16,1000.14,499.96,0.0,...,10.8745,10.9010,10.8717,10.8809,-0.0201,-0.0101,NaN,1.870000e-08,1.820000e-08,2023-01-13 17:49:34.461000
8930,13/01/2023,17:49:35.662,10720.253,499.96,100.08,1000.02,1000.16,1000.14,499.96,0.0,...,10.8745,10.9010,10.8718,10.8822,-0.0201,-0.0101,NaN,1.870000e-08,1.820000e-08,2023-01-13 17:49:35.662000
8931,13/01/2023,17:49:36.865,10721.456,499.96,100.08,1000.02,1000.16,1000.14,499.96,0.0,...,10.8745,10.9003,10.8710,10.8807,-0.0201,-0.0101,NaN,1.870000e-08,1.820000e-08,2023-01-13 17:49:36.865000
8932,13/01/2023,17:49:38.752,10723.343,429.64,100.08,1000.02,1000.16,1000.14,499.96,0.0,...,10.8745,10.9003,10.8709,10.8802,-0.0201,-0.0101,NaN,-4.323000e-07,1.820000e-08,2023-01-13 17:49:38.752000


### Data processing

In [5]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [6]:
#Filtering background signal

#Find the timestamp where emission was turned on and off
print("Result 1:",hv.loc[hv["Comment"]=="grid transparent"])
print("Result 2:", hv.loc[hv["Comment"]=="grid repelling"])
print("Result 3:", hv.loc[hv["Comment"]=="electron activity grid repelling"])

x=pd.Timestamp(2023, 1, 12, 17 ,23, 58)
y=pd.Timestamp(2023, 1, 12, 17, 28, 43)
z=pd.Timestamp(2023, 1, 12, 17, 20, 13)
#then find the time value closest to it in the MID scan file 
print("\n","Result 4:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])
print("\n","Result 5:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - y).abs().argsort(),:])
print("\n","Result 6:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - z).abs().argsort(),:])

#take a mass with a signal known to be in the background
bck=np.mean(mid["mass 40.0_value"])

#apply background correction to the rest of the mass spectrum
mid["mass 2, corr"]=mid["mass 2.0_value"]-bck

 
#From the signal, get the partial pressures 
h2_sens=2.13e-2
p_h2=mid["mass 2, corr"][91:136]/h2_sens
print(p_h2.describe())

#baseline for partial pressure delta
base_h2=mid["mass 2, corr"][56:91]/h2_sens
base_h2.describe()

Result 1:            Date          Time  Elapsed time  VMon1   VMon2   VMon3   VMon4  \
196  13/01/2023  14:54:55.693       240.284  82.72  100.00  999.94  999.98   
197  13/01/2023  14:54:56.886       241.477   0.88  100.02  999.92  999.96   
198  13/01/2023  14:54:58.074       242.665   0.22  100.02  999.92  999.96   
199  13/01/2023  14:54:59.269       243.860   0.18  100.02  999.92  999.96   

      VMon5   VMon6  VMon7  ...   IMon3   IMon4   IMon5   IMon6   IMon7  \
196  999.90  499.86    0.0  ...  6.9864  7.0585  7.0920  7.1649 -0.0218   
197  999.90  499.86    0.0  ...  7.1135  7.1710  7.1058  7.1242 -0.0218   
198  999.92  499.86    0.0  ...  7.1913  7.1925  7.1905  7.2062 -0.0218   
199  999.92  499.86    0.0  ...  7.1995  7.2069  7.1929  7.2059 -0.0218   

      IMon8           Comment       hv_grid            em  \
196 -0.0139  grid transparent -4.289000e-07  6.707000e-07   
197 -0.0139  grid transparent -3.288000e-07  1.931600e-06   
198 -0.0139  grid transparent -3.088000e

count    3.500000e+01
mean     1.693038e-08
std      1.990342e-10
min      1.660105e-08
25%      1.675828e-08
50%      1.695058e-08
75%      1.709023e-08
max      1.727560e-08
Name: mass 2, corr, dtype: float64

### Injection pressure linearity

In [7]:
#defining a function for the straight line
def func(x,a,b):
    return a*x + b
#curve fit for the data
y = vaclog["injection 100mbar"].values
x = (vaclog["Elapsed time"]/3600).values
params, cov = curve_fit(func,x,y)
#straight line parameters
a, b = params
print(f"Fitted line: y = {a:.2f}*x+{b:.2f}")

Fitted line: y = 0.59*x+81.42


In [8]:
#checking inj pressure linearity, graph
fig, ax = plt.subplots()
y_fit = func(x,a,b)
ax.plot((vaclog["Elapsed time"]/3600),vaclog["injection 100mbar"],marker=".", markersize=5,label='inj volume pressure')
ax.plot((vaclog["Elapsed time"]/3600),y_fit, "--",color="red", label="Y= -ax + b fit") 
ax.legend(loc="upper right")

ax.set_xlabel('Elapsed time (h)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')
plt.title("Injection pressure evolution 100mbar conductance gauge")

Text(0.5, 1.0, 'Injection pressure evolution 100mbar conductance gauge')

## Point-by-point desorption

In [9]:
#make a sub-df from mid that includes only mass 2 data
h2 = mid.loc[:, ["mass 2.0_time", "mass 2.0_value"]]
#rename columns
h2 = h2.rename(columns={"mass 2.0_time": "datetime","mass 2.0_value": "Ion current"}).reset_index(drop=True)

print(h2)

                     datetime   Ion current
0     2023/01/13 14:48:15.864  3.501160e-10
1     2023/01/13 14:48:22.260  3.574470e-10
2     2023/01/13 14:48:28.655  3.623710e-10
3     2023/01/13 14:48:35.050  3.698370e-10
4     2023/01/13 14:48:41.445  3.766090e-10
...                       ...           ...
1692  2023/01/13 17:48:39.246  2.837350e-11
1693  2023/01/13 17:48:45.641  2.830790e-11
1694  2023/01/13 17:48:52.036  2.827990e-11
1695  2023/01/13 17:48:58.431  2.821060e-11
1696  2023/01/13 17:49:04.826  2.820090e-11

[1697 rows x 2 columns]


In [10]:
# Convert the 'time' column to datetime index for both dataframes
hv.index = (pd.to_datetime(hv['datetime']).dt.round(freq="S"))
h2.index = (pd.to_datetime(h2['datetime']).dt.round(freq="S"))
print(hv)

# Reindex df1 to match the datetime index of df2
hv = hv.reindex(h2.index, method='nearest', limit=1)

#Drop the "datetime" columns
hv.drop('datetime', axis=1, inplace=True)
h2.drop('datetime', axis=1, inplace=True)

# Now df1 and df2 have the same datetime index and can be joined or aligned
# by joining on the index
hv_aligned = hv.merge(h2,left_index=True, right_index=True, how="inner")

print("aligned",hv_aligned)
print(hv_aligned.describe())

                           Date          Time  Elapsed time   VMon1   VMon2  \
datetime                                                                      
2023-01-13 14:50:55  13/01/2023  14:50:55.409         0.000    0.02    0.06   
2023-01-13 14:50:57  13/01/2023  14:50:56.594         1.185    0.02    0.06   
2023-01-13 14:50:58  13/01/2023  14:50:57.767         2.358    0.02    0.06   
2023-01-13 14:50:59  13/01/2023  14:50:58.953         3.544    0.02    0.06   
2023-01-13 14:51:00  13/01/2023  14:51:00.135         4.726    0.02    0.06   
...                         ...           ...           ...     ...     ...   
2023-01-13 17:49:34  13/01/2023  17:49:34.461     10719.052  499.96  100.08   
2023-01-13 17:49:36  13/01/2023  17:49:35.662     10720.253  499.96  100.08   
2023-01-13 17:49:37  13/01/2023  17:49:36.865     10721.456  499.96  100.08   
2023-01-13 17:49:39  13/01/2023  17:49:38.752     10723.343  429.64  100.08   
2023-01-13 17:49:40  13/01/2023  17:49:39.949     10

In [11]:
#Filtering background signal for interpolated datasets

#take a mass with a signal known to be in the background
bck = np.mean(mid["mass 5.5_value"])

#apply background correction to the rest of the mass spectrum
hv_aligned["mass 2 corr"] = hv_aligned["Ion current"]-bck

#From the signal, get the partial pressures 
h2_sens = 2.13e-2 #A/mbar
p_h2_ip = hv_aligned["mass 2 corr"]/(h2_sens*0.46) #H2 equivalent pressures

#account for the thermal transpiration factor
T2 = 4.2
T1 = 296.5
T_coef = np.sqrt(T2/T1)
p_h2_ip = p_h2_ip*T_coef 

#insert partial pressures into the dataframe
hv_aligned.insert(22,"hydrogen pp",p_h2_ip)


In [12]:
hv_aligned.fillna(0,inplace=True)
hv_aligned["hydrogen pp"].describe()

count    1.697000e+03
mean     2.473906e-09
std      6.402917e-09
min      3.418474e-10
25%      5.021827e-10
50%      6.250925e-10
75%      3.065728e-09
max      6.550687e-08
Name: hydrogen pp, dtype: float64

# Primary desorption calculation method
### Elena's formula for the cryo setup
$$
\eta=\frac{C_{H2}\cdot (\Delta p_1 - \Delta p_2)q_e}{k_B\cdot T \cdot I_e}
$$

Where $\Delta p_i=p(t)_i - p(t=0)_i$ and $i=1,2$
In the case of the 4K setup, we know the exact conductance of gasses. The formula above applies well.

Due to the EGA being fully retracted, it is safe to assume that the desorption takes place at approximately room temperature

In [13]:
#calculation delta_p-s
dp1 = (np.mean(p_h2)-np.mean(base_h2))*100
print(dp1)

#Constants
k_B = 1.38e-23
T = 293
print(T)
q_e = 1.6e-19
c_h2 = 37.91/1000
s_h2=42/1000
x_h2=1/((1/c_h2)+(1/s_h2))
print(x_h2)
#Emission current

I_em = np.nanmean(hv["em"].nlargest(30))
print(I_em)
#Conductance for CO in m3/s
c_co=10.13/1000
c_co2 = 8.08/1000


-7.189471644682957e-08
293
0.01992516581153798
3.517576666666666e-06


In [14]:
#Desorption yield
eta_h2 = (x_h2*dp1*q_e)/(k_B*T*I_em)
#eta_co = (c_co*(dp1-dp2)*q_e)/(k_B*T*I_em)
print("The desorption yield for H2 [mol/el]:",eta_h2,"\n",)#""The desorption yield for CO2 [mol/el]:",eta_co2)

The desorption yield for H2 [mol/el]: -0.016114937424631572 



## Desorption yield point-by-point

$$
\eta=\frac{C_{H2}\cdot (p_{n} - p_{n-1}) \cdot q_e}{k_B\cdot T \cdot I_{n}}
$$


In [15]:
#partial pressure delta

p_delta = hv_aligned["hydrogen pp"].diff().fillna(hv_aligned["hydrogen pp"])*100
p_delta = p_delta.clip(lower=0)
print(p_delta)
#hv_shifted = hv_aligned.shift(periods=1)
#print(hv_shifted)
#p_delta = hv_aligned["hydrogen pp"] - hv_shifted["hydrogen pp"]
#print(hv_aligned["hydrogen pp"].describe())
#print(hv_shifted["hydrogen pp"].describe())
print(p_delta.describe())

datetime
2023-01-13 14:48:16    4.252203e-07
2023-01-13 14:48:22    8.905085e-09
2023-01-13 14:48:29    5.981263e-09
2023-01-13 14:48:35    9.069072e-09
2023-01-13 14:48:41    8.226059e-09
                           ...     
2023-01-13 17:48:39    0.000000e+00
2023-01-13 17:48:46    0.000000e+00
2023-01-13 17:48:52    0.000000e+00
2023-01-13 17:48:58    0.000000e+00
2023-01-13 17:49:05    0.000000e+00
Name: hydrogen pp, Length: 1697, dtype: float64
count    1.697000e+03
mean     4.214024e-09
std      3.497321e-08
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      9.669142e-11
max      6.158247e-07
Name: hydrogen pp, dtype: float64


In [16]:
#constants are already defined
#emission current and H2 partial pressures are now available at the same sampling rates
print(hv_aligned.describe())
I_n = hv_aligned["em"].replace(0, np.nan).dropna()

print(I_n)
#I_n.where(em <= 1.5e-6, np.nan, inplace=True)

eta = ((x_h2*p_delta*q_e)/(k_B*T*I_n)).dropna()
eta = eta.loc[eta.index[1]:]
print("desorption yields point-by-point:","\n", eta)

       Elapsed time        VMon1        VMon2        VMon3        VMon4  \
count   1697.000000  1697.000000  1697.000000  1697.000000  1697.000000   
mean    5266.701038   137.804290    97.628874   974.787331   974.721155   
std     3132.840375   223.176881    15.392420   156.480920   157.062546   
min        0.000000     0.000000     0.000000     0.000000     0.000000   
25%     2552.128000     0.100000   100.060000  1000.020000  1000.140000   
50%     5266.477000     0.120000   100.060000  1000.020000  1000.160000   
75%     7977.590000   499.940000   100.080000  1000.040000  1000.160000   
max    10690.185000   499.980000   100.200000  1000.040000  1000.180000   

             VMon5        VMon6   VMon7        VMon8        IMon1  ...  \
count  1697.000000  1697.000000  1697.0  1697.000000  1697.000000  ...   
mean    972.667331   484.938291     0.0     0.020837    -0.277369  ...   
std     160.136540    85.385606     0.0     0.005231     0.197268  ...   
min       0.000000     0.000

In [17]:
#slicing eta to match hv_aligned["Elapsed time"]:
# Slice x using the index of y
hv_downsampled = hv_aligned.loc[eta.index, :]

# Plotting

In [18]:
#plot the aligned dataframe results
plt.figure(figsize=(12,6))
plt.plot(hv_aligned["Elapsed time"],hv_aligned["Ion current"],linestyle="-",marker=".", markersize=4,label='H2 ion current')
plt.plot(hv_aligned["Elapsed time"],hv_aligned["em"],linestyle="-",marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("Current evolution 1 ML tech. surface pre-injection, EGA fully in, dynamic vacuum")


plt.yscale('log')


In [19]:
#plot the h2 mid 
#seems to be too little rows?
plt.figure(figsize=(12,6))
plt.plot(hv_aligned["Elapsed time"],hv_aligned["Ion current"],linestyle="-",marker=".", markersize=4,label='H2 ion current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("Current evolution 1 ML tech. surface pre-injection, EGA fully in, dynamic vacuum")


plt.yscale('linear')


In [20]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(14,8))
plt.plot(time['Elapsed time, mass 2'],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(time['Elapsed time, mass 4'],mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(time['Elapsed time, mass 12'],mid["mass 12.0_value"],marker=".", markersize=4,label='M14')
plt.plot(time['Elapsed time, mass 16'],mid["mass 16.0_value"],marker=".", markersize=4,label='M16')
plt.plot(time['Elapsed time, mass 18'],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(time['Elapsed time, mass 28'],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(time['Elapsed time, mass 32'],mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(time['Elapsed time, mass 40'],mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
plt.plot(time["Elapsed time, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("MID scan 10 ML tech. surface pre-injection, EGA fully in, dynamic vacuum")

plt.yscale('log')

In [21]:
#plotting HiVolta log
plt.figure(figsize=(12,6))
plt.plot(hv["Elapsed time"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["Elapsed time"],hv["em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("Current evolution \n ~100 ML tech. surface pre-injection, EGA fully in, dynamic vacuum")


plt.yscale('log')


In [27]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(10,6))
plt.plot(vaclog["Elapsed time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Total pressure evolution \n ~100 ML tech. surface pre-injection, EGA fully in, dynamic vacuum")
ax = plt.gca()
ax.set_yscale('log')
plt.tick_params(axis="y", which='minor')
plt.grid(visible=None, which='both', axis='both')
plt.show()

In [23]:
## Plotting the temp evolution
plt.figure()
plt.plot((vaclog["Elapsed time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("Temperature evolution \n ~100 ML tech. surface pre-injection, EGA fully in, dynamic vacuum")
plt.yscale('linear')

In [24]:
#plotting point-by-point eta
plt.figure(figsize=(10,6))
plt.plot(hv_downsampled["Elapsed time"],eta,marker=".", markersize=4,label='eta')
#plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Electon stimulated desorption yield (M/el)')
plt.legend()
plt.title("ESD vs time \n ~100 ML tech. surface pre-injection, EGA fully in, dynamic vacuum")
ax = plt.gca()
ax.set_yscale('log')
ax.grid(which="major",axis="x")
plt.tick_params(axis="y", which='minor')
plt.grid(visible=None, which='both', axis='both')
plt.show()